In [1]:
import sys
import os
sys.path.append(os.path.join(r'D:\e-commerce_env\e-commerce\Lib\site-packages'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data.csv',encoding= 'unicode_escape')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [8]:
df['StockCode'].value_counts()

85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
          ... 
21431        1
22275        1
17001        1
90187A       1
72759        1
Name: StockCode, Length: 4070, dtype: int64

In [2]:
def transform_datatypes_and_nulls(df):
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    df['CustomerID'].fillna(0,inplace=True)
    df['CustomerID'] = df['CustomerID'].astype(int)
    df['Quantity'] = df['Quantity'].astype(int)

def fill_null_values_in_descripctions(df):
    # Sort the DataFrame by 'StockCode' and 'InvoiceDate' to ensure rows with the same 'StockCode' are consecutive
    df.sort_values(by=['StockCode', 'InvoiceDate'], inplace=True)
    # Use forward-fill (ffill) to replace null values in 'Description' column
    df['Description'].fillna(method='ffill', inplace=True)
    
# all transactions which weren't orders normal orders, rather some fee payments etc.
def divide_uncommon_transactions(df):
    df_M = df[df['StockCode'] == 'M']
    df_DOT = df[df['StockCode'] == 'DOT']
    df_BANK_CHARGES = df[df['StockCode'] == 'BANK CHARGES']
    df_AMAZONFEE = df[df['StockCode'] == 'AMAZONFEE']
    df_BAD_DEBT = df[df['StockCode'] == "B"]
    df_POSTAGE = df[df['StockCode'] == "POST"]
    df_DISCOUNT = df[df['StockCode'] == "D"]
    df_CRUK_COMMISIONS = df[df['StockCode'] == "CRUK"]

    return {
        'M': df_M,
        'DOT': df_DOT,
        'BANK CHARGES': df_BANK_CHARGES,
        'AMAZONFEE': df_AMAZONFEE,
        'B': df_BAD_DEBT,
        'POST': df_POSTAGE,
        'D': df_DISCOUNT,
        'CRUK': df_CRUK_COMMISIONS
    }

def create_filtered_orders(df):
    mask = (df['StockCode'] != "AMAZONFEE") & (df['StockCode'] != "BANK CHARGES") & (df['StockCode'] != "M") & (df['StockCode'] != "DOT") & (df['StockCode'] != "B")& (df['StockCode'] != "POST") & (df['StockCode'] != "D") &(df['StockCode'] != "CRUK")
    filtered_df = df[mask]
    return filtered_df

# checking for all canceled orders (9288 orders)
def cancelled_orders(df):
    mask = df['InvoiceNo'].str.startswith('C')
    cancelled_orders = df[mask]
    return cancelled_orders

#items which are damaged, lost but they are not returns
def items_which_are_not_sellable():
    maska = filtered_df['InvoiceNo'].str.startswith('C')
    not_sellable_items = filtered_df[(maska==False) & (filtered_df['Quantity']<0)]
    return not_sellable_items

def remove_cancelled_lost_and_damaged_orders_from_filtered_df():
    new_df = filtered_df[(filtered_df['Quantity']<0)==False]
    return new_df

def create_df_with_manual_returns():
    df_M_returns = df_M[df_M['Quantity']<0]
    return df_M_returns
def add_full_items_price_to_df(df):
    df['FullPrice'] = df['Quantity'] * df['UnitPrice'] 
    
df = pd.read_csv('data.csv',encoding= 'unicode_escape')

transform_datatypes_and_nulls(df)
fill_null_values_in_descripctions(df)
df.sort_values(by=['InvoiceDate','InvoiceNo'],inplace=True)
df = df.reset_index(drop=True)

# Call the function and store the results in a dictionary
category_dataframes = divide_uncommon_transactions(df)

# Access each category's DataFrame
df_M = category_dataframes['M']
df_DOT = category_dataframes['DOT']
df_BANK_CHARGES = category_dataframes['BANK CHARGES']
df_AMAZONFEE = category_dataframes['AMAZONFEE']
df_BAD_DEBT = category_dataframes['B']
df_POSTAGE = category_dataframes['POST']
df_DISCOUNT = category_dataframes['D']
df_CRUK_COMMISIONS = category_dataframes['CRUK']
df_M_returns = create_df_with_manual_returns()
df_M = df_M[df_M['Quantity']>=0]

filtered_df = create_filtered_orders(df)
filtered_df = filtered_df.reset_index(drop=True)
cancelled_orders_and_returns = cancelled_orders(filtered_df)
not_sellable_items = items_which_are_not_sellable()
filtered_df = remove_cancelled_lost_and_damaged_orders_from_filtered_df()
add_full_items_price_to_df(filtered_df)
#concat all returns and cancelled orders
add_full_items_price_to_df(cancelled_orders_and_returns) 
# using abs() on full price and adding AbsQuantity, so I'm being able to match it with orders from filtered_df
cancelled_orders_and_returns['FullPrice'] = cancelled_orders_and_returns['FullPrice'].apply(lambda x: abs(x))
cancelled_orders_and_returns['Quantity'] = cancelled_orders_and_returns['Quantity'].apply(lambda x: abs(x))

In [16]:
filtered_df1

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FullPrice
5,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
12,536367,22310,IVORY KNITTED MUG COSY,6,2010-12-01 08:34:00,1.65,13047,United Kingdom,9.90
17,536367,22749,FELTCRAFT PRINCESS CHARLOTTE DOLL,8,2010-12-01 08:34:00,3.75,13047,United Kingdom,30.00
19,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom,54.08
26,536370,10002,INFLATABLE POLITICAL GLOBE,48,2010-12-01 08:45:00,0.85,12583,France,40.80
...,...,...,...,...,...,...,...,...,...
539200,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09 12:50:00,3.75,12680,France,15.00
539201,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
539202,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
539203,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [11]:
filtered_df[filtered_df['CustomerID']==17548]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FullPrice
164164,550755,22079,RIBBON REEL HEARTS DESIGN,10,2011-04-20 12:01:00,1.65,17548,United Kingdom,16.5
164165,550755,22081,RIBBON REEL FLORA + FAUNA,10,2011-04-20 12:01:00,1.65,17548,United Kingdom,16.5
164166,550755,22082,RIBBON REEL STRIPES DESIGN,10,2011-04-20 12:01:00,1.65,17548,United Kingdom,16.5
164167,550755,22585,PACK OF 6 BIRDY GIFT TAGS,24,2011-04-20 12:01:00,1.25,17548,United Kingdom,30.0
164168,550755,22926,IVORY GIANT GARDEN THERMOMETER,4,2011-04-20 12:01:00,5.95,17548,United Kingdom,23.8


In [24]:
maska = filtered_df['InvoiceNo'].str.startswith('C')
filtered_df[(maska==False) & (filtered_df['Quantity']<0)]

0    1336
Name: CustomerID, dtype: int64

In [17]:
cancelled_orders_and_returns[cancelled_orders_and_returns['CustomerID']==17548]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,FullPrice,AbsQuantity
1,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548,United Kingdom,41.40,12
2,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom,6.96,24
3,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom,6.96,24
4,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom,6.96,24
5,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548,United Kingdom,39.60,24
6,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom,19.80,12
7,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom,19.80,12
3093,C552049,22079,RIBBON REEL HEARTS DESIGN,-10,2011-05-06 10:00:00,1.65,17548,United Kingdom,16.50,10
3094,C552049,22081,RIBBON REEL FLORA + FAUNA,-10,2011-05-06 10:00:00,1.65,17548,United Kingdom,16.50,10
3095,C552049,22082,RIBBON REEL STRIPES DESIGN,-10,2011-05-06 10:00:00,1.65,17548,United Kingdom,16.50,10


In [16]:
df[df['CustomerID']==17548]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
235,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548,United Kingdom
236,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
238,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
239,C536391,22553,PLASTERS IN TIN SKULLS,-24,2010-12-01 10:24:00,1.65,17548,United Kingdom
240,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
241,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
165024,550755,22079,RIBBON REEL HEARTS DESIGN,10,2011-04-20 12:01:00,1.65,17548,United Kingdom
165025,550755,22081,RIBBON REEL FLORA + FAUNA,10,2011-04-20 12:01:00,1.65,17548,United Kingdom
165026,550755,22082,RIBBON REEL STRIPES DESIGN,10,2011-04-20 12:01:00,1.65,17548,United Kingdom


In [4]:
cancelled_orders_and_returns

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
1,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548,United Kingdom
2,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
3,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
4,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
...,...,...,...,...,...,...,...,...
9006,C580263,M,Manual,-16,2011-12-02 12:43:00,0.29,12536,France
9007,C580510,M,Manual,-1,2011-12-04 13:48:00,1.25,14456,United Kingdom
9008,C581009,M,Manual,-1,2011-12-07 09:15:00,125.00,16971,United Kingdom
9009,C581145,M,Manual,-1,2011-12-07 13:48:00,9.95,17490,United Kingdom


In [21]:
df_M_returns = df_M[df_M['Quantity']<0]

In [9]:
df_M.sort_values(by=['UnitPrice'],ascending=False).head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
222681,C556445,M,Manual,-1,2011-06-10 15:31:00,38970.00,15098,United Kingdom
173391,C551699,M,Manual,-1,2011-05-03 14:12:00,6930.00,16029,United Kingdom
268028,560373,M,Manual,1,2011-07-18 12:30:00,4287.63,0,United Kingdom
268027,C560372,M,Manual,-1,2011-07-18 12:26:00,4287.63,17448,United Kingdom
422375,C573079,M,Manual,-2,2011-10-27 14:15:00,4161.06,12536,France
422376,573080,M,Manual,1,2011-10-27 14:20:00,4161.06,12536,France
422351,573077,M,Manual,1,2011-10-27 14:13:00,4161.06,12536,France
406406,571751,M,Manual,1,2011-10-19 11:18:00,3949.32,12744,Singapore
406404,C571750,M,Manual,-1,2011-10-19 11:16:00,3949.32,12744,Singapore
293842,C562647,M,Manual,-1,2011-08-08 12:56:00,3155.95,15502,United Kingdom


In [11]:
maska = cancelled_orders_and_returns['InvoiceNo'].str.startswith('C')==False
cancelled_orders_and_returns[maska]['Quantity'].value_counts()

-1       79
-2       58
-5       46
-3       39
-6       36
         ..
-116      1
-304      1
-272      1
-1206     1
-235      1
Name: Quantity, Length: 298, dtype: int64

In [21]:
filtered_df[filtered_df['InvoiceNo']=='536996']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
7165,536996,22712,CARD DOLLY GIRL,-20,2010-12-03 15:30:00,0.0,0,United Kingdom


In [5]:
df_POSTAGE.sort_values(by=['UnitPrice'])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
52262,540699,POST,POSTAGE,1000,2011-01-11 09:32:00,0.00,0,United Kingdom
326552,565556,POST,POSTAGE,750,2011-09-05 12:14:00,0.00,0,United Kingdom
205991,554857,POST,POSTAGE,800,2011-05-27 10:08:00,0.00,0,United Kingdom
453998,575505,POST,POSTAGE,800,2011-11-10 10:29:00,0.00,0,United Kingdom
431374,573589,POST,POSTAGE,1,2011-10-31 15:11:00,0.55,0,United Kingdom
...,...,...,...,...,...,...,...,...
235379,C557638,POST,POSTAGE,-1,2011-06-21 16:01:00,545.58,17450,United Kingdom
264976,560187,POST,POSTAGE,1,2011-07-15 12:05:00,550.94,17444,Canada
236402,557754,POST,POSTAGE,1,2011-06-22 13:12:00,700.00,12432,Norway
173277,C551685,POST,POSTAGE,-1,2011-05-03 12:51:00,8142.75,16029,United Kingdom


In [28]:
cancelled_orders_and_returns[cancelled_orders_and_returns['CustomerID']==14527]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
148506,C549288,22469,HEART OF WICKER SMALL,-1,2011-04-07 18:06:00,1.65,14527,United Kingdom
148507,C549288,22788,BROCANTE COAT RACK,-1,2011-04-07 18:06:00,9.95,14527,United Kingdom
224637,C556735,22169,FAMILY ALBUM WHITE PICTURE FRAME,-2,2011-06-14 12:01:00,8.50,14527,United Kingdom


In [25]:
df_DISCOUNT

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
9038,C537164,D,Discount,-1,2010-12-05 13:21:00,29.29,14527,United Kingdom
14498,C537597,D,Discount,-1,2010-12-07 12:34:00,281.00,15498,United Kingdom
19392,C537857,D,Discount,-1,2010-12-08 16:00:00,267.12,17340,United Kingdom
31134,C538897,D,Discount,-1,2010-12-15 09:14:00,5.76,16422,United Kingdom
...,...,...,...,...,...,...,...,...
479868,C577227,D,Discount,-1,2011-11-18 12:06:00,19.82,14527,United Kingdom
479869,C577227,D,Discount,-1,2011-11-18 12:06:00,16.76,14527,United Kingdom
493613,C578239,D,Discount,-1,2011-11-23 12:29:00,26.33,14912,Italy
516221,C579884,D,Discount,-1,2011-11-30 17:34:00,20.53,14527,United Kingdom


In [11]:
quantity_stats = filtered_df['Quantity'].describe()
unit_price_stats = filtered_df['UnitPrice'].describe()
print("Quantity Statistics:")
print(quantity_stats)

print("\nUnitPrice Statistics:")
print(unit_price_stats)

Quantity Statistics:
count    529102.000000
mean         10.670014
std         157.096419
min           1.000000
25%           1.000000
50%           3.000000
75%          11.000000
max       80995.000000
Name: Quantity, dtype: float64

UnitPrice Statistics:
count    529102.000000
mean          3.271793
std           4.446915
min           0.000000
25%           1.250000
50%           2.080000
75%           4.130000
max         649.500000
Name: UnitPrice, dtype: float64


In [11]:
filtered_df[filtered_df['Quantity']==filtered_df['Quantity'].max()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
537725,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446,United Kingdom


In [12]:
filtered_df[filtered_df['Quantity']==filtered_df['Quantity'].min()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
537726,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446,United Kingdom


In [11]:
filtered_df['StockCode'].value_counts()

85123A    2313
22423     2203
85099B    2159
47566     1727
20725     1639
          ... 
85160a       1
62095B       1
85018C       1
84670        1
21653        1
Name: StockCode, Length: 4062, dtype: int64

In [12]:
filtered_df[(filtered_df['InvoiceDate']>'2010-12-01 09:00:00') & (filtered_df['InvoiceDate']<'2010-12-01 09:50:00') & (filtered_df['CustomerID']==15311)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
105,536381,15056BL,EDWARDIAN PARASOL BLACK,2,2010-12-01 09:41:00,5.95,15311,United Kingdom
106,536381,15056N,EDWARDIAN PARASOL NATURAL,2,2010-12-01 09:41:00,5.95,15311,United Kingdom
107,536381,21166,COOK WITH WINE METAL SIGN,1,2010-12-01 09:41:00,1.95,15311,United Kingdom
108,536381,21169,YOU'RE CONFUSING ME METAL SIGN,3,2010-12-01 09:41:00,1.69,15311,United Kingdom
109,536381,21175,GIN + TONIC DIET METAL SIGN,2,2010-12-01 09:41:00,2.10,15311,United Kingdom
110,536381,21523,DOORMAT FANCY FONT HOME SWEET HOME,10,2010-12-01 09:41:00,6.75,15311,United Kingdom
111,536381,21533,RETROSPOT LARGE MILK JUG,1,2010-12-01 09:41:00,4.95,15311,United Kingdom
112,536381,21557,SET OF 6 FUNKY BEAKERS,2,2010-12-01 09:41:00,2.95,15311,United Kingdom
113,536381,21672,WHITE SPOT RED CERAMIC DRAWER KNOB,6,2010-12-01 09:41:00,1.25,15311,United Kingdom
114,536381,21731,RED TOADSTOOL LED NIGHT LIGHT,2,2010-12-01 09:41:00,1.65,15311,United Kingdom


In [61]:
filtered_df[filtered_df['InvoiceNo'].str.startswith('C')==True]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
...,...,...,...,...,...,...,...,...
540448,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397,United Kingdom
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom


In [ ]:
filtered_df

In [46]:
cancelled_orders

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom


In [44]:
filtered_df[filtered_df['InvoiceNo']=='C536379']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [8]:
# countries which buy the most 

def create_country_orders_tuples():
    country_order_counts = filtered_df['Country'].value_counts()
    country_order_tuples = [(country, count) for country, count in country_order_counts.items() if (country !='Unspecified') and (country!= 'Channel Islands')]
    return country_order_tuples

def map_countries_with_codes(country_order_tuples):

    country_codes = {
        'United Kingdom': 'gb',
        'Germany': 'de',
        'France': 'fr',
        'EIRE': 'ie',
        'Spain': 'es',
        'Netherlands': 'nl',
        'Belgium': 'be',
        'Switzerland': 'ch',
        'Portugal': 'pt',
        'Australia': 'au',
        'Norway': 'no',
        'Italy': 'it',
        'Finland': 'fi',
        'Cyprus': 'cy',
        'Sweden': 'se',
        'Austria': 'at',
        'Denmark': 'dk',
        'Japan': 'jp',
        'Poland': 'pl',
        'Israel': 'il',
        'USA': 'us',
        'Hong Kong': 'hk',
        'Singapore': 'sg',
        'Iceland': 'is',
        'Canada': 'ca',
        'Greece': 'gr',
        'Malta': 'mt',
        'United Arab Emirates': 'ae',
        'European Community': 'eu',
        'RSA': 'za',
        'Lebanon': 'lb',
        'Lithuania': 'lt',
        'Brazil': 'br',
        'Czech Republic': 'cz',
        'Bahrain': 'bh',
        'Saudi Arabia': 'sa'
    }   
    country_order_tuples_with_codes = [
        (country_codes.get(country, country), order) for country, order in country_order_tuples
    ]  
    return country_order_tuples_with_codes

In [45]:
# values wich can't be showed on the map
unsepcifed_country = [('unspecified',446)]
channel_islands_orders = [('Channel Islands',756)]

country_order_tuples = create_country_orders_tuples()
country_order_tuples_with_codes = map_countries_with_codes(country_order_tuples)

In [50]:
# I removed UK which is clearly ahead of the competition :D
country_order_tuples_with_codes_without_uk = [('de', 9096),
 ('fr', 8236),
 ('ie', 8183),
 ('es', 2468),
 ('nl', 2330),
 ('be', 1971),
 ('ch', 1969),
 ('pt', 1475),
 ('au', 1257),
 ('no', 1060),
 ('it', 783),
 ('fi', 653),
 ('cy', 619),
 ('se', 437),
 ('at', 387),
 ('dk', 375),
 ('jp', 355),
 ('pl', 336),
 ('il', 297),
 ('us', 291),
 ('hk', 280),
 ('sg', 215),
 ('is', 182),
 ('ca', 150),
 ('gr', 142),
 ('mt', 123),
 ('ae', 67),
 ('eu', 58),
 ('za', 57),
 ('lb', 45),
 ('lt', 35),
 ('br', 32),
 ('cz', 28),
 ('bh', 19),
 ('sa', 10)]

In [49]:
from pygal_maps_world.maps import World
def create_interactive_map_and_save_as_file():
    worldmap_chart = World()
    worldmap_chart.title = 'Number of orders based on Country'
    worldmap_chart.add('In 2010-2011',country_order_tuples_with_codes_without_uk)
    worldmap_chart.render_to_file('countries_map.svg')

create_interactive_map_and_save_as_file()

In [51]:
grouped_by_customer = filtered_df.groupby('CustomerID')

In [53]:
grouped_by_customer
average_order_size_per_customer = grouped_by_customer['Quantity'].mean()

In [61]:
average_order_size_per_customer.sort_values()

CustomerID
17307     -144.000000
16742      -63.333333
14213      -48.800000
12666      -28.000000
15638      -26.000000
             ...     
14609     1756.500000
16308     2000.000000
16754     2140.000000
13135     4300.000000
13256    12540.000000
Name: Quantity, Length: 4364, dtype: float64

In [25]:
filtered_df[filtered_df['Quantity']<0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
152,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
233,C536391,21484,CHICK GREY HOT WATER BOTTLE,-12,2010-12-01 10:24:00,3.45,17548,United Kingdom
234,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
235,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
...,...,...,...,...,...,...,...,...
537752,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397,United Kingdom
537753,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397,United Kingdom
539014,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
539015,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315,United Kingdom
